In [1]:
!curl -o text.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 82 1089k   82  894k    0     0  1304k      0 --:--:-- --:--:-- --:--:-- 1306k
100 1089k  100 1089k    0     0  1530k      0 --:--:-- --:--:-- --:--:-- 1534k


In [1]:
file = open("text.txt", 'r')
text = file.read()
file.close()

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
stoi = {c:i for i, c in enumerate(chars, start=0)}
itos = {i:c for i, c in enumerate(chars, 0)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.dtype, data.shape)
print(data[:10])

torch.int64 torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [5]:
n = int(0.9*(len(data)))
train = data[:n]
val = data[n:]

In [6]:
block_size = 8
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [7]:
x = train[:block_size]
y = train[1:block_size+1]
for i in range(1, block_size):
    print("context is", x[:i], "target is", y[i-1])

context is tensor([18]) target is tensor(47)
context is tensor([18, 47]) target is tensor(56)
context is tensor([18, 47, 56]) target is tensor(57)
context is tensor([18, 47, 56, 57]) target is tensor(58)
context is tensor([18, 47, 56, 57, 58]) target is tensor(1)
context is tensor([18, 47, 56, 57, 58,  1]) target is tensor(15)
context is tensor([18, 47, 56, 57, 58,  1, 15]) target is tensor(47)


In [8]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    data = train if split=="train" else val
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

# example usage
xe, ye = get_batch(split="train")
for i in range(batch_size):
    for j in range(block_size):
        print("context:", xe[i][:j+1], "target:", ye[i][j])

context: tensor([24]) target: tensor(43)
context: tensor([24, 43]) target: tensor(58)
context: tensor([24, 43, 58]) target: tensor(5)
context: tensor([24, 43, 58,  5]) target: tensor(57)
context: tensor([24, 43, 58,  5, 57]) target: tensor(1)
context: tensor([24, 43, 58,  5, 57,  1]) target: tensor(46)
context: tensor([24, 43, 58,  5, 57,  1, 46]) target: tensor(43)
context: tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: tensor(39)
context: tensor([44]) target: tensor(53)
context: tensor([44, 53]) target: tensor(56)
context: tensor([44, 53, 56]) target: tensor(1)
context: tensor([44, 53, 56,  1]) target: tensor(58)
context: tensor([44, 53, 56,  1, 58]) target: tensor(46)
context: tensor([44, 53, 56,  1, 58, 46]) target: tensor(39)
context: tensor([44, 53, 56,  1, 58, 46, 39]) target: tensor(58)
context: tensor([44, 53, 56,  1, 58, 46, 39, 58]) target: tensor(1)
context: tensor([52]) target: tensor(58)
context: tensor([52, 58]) target: tensor(1)
context: tensor([52, 58,  1]) target: t

In [ ]:
import torch
import torch.nn as nn
import torch.functional as F
class BigramLanguageModel(nn.module):
    def __init__(self, vocab_size, embedding_dim):
        super.__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, embedding_dim)
    def forward(self, idx, target):
        logits = self.token_embedding_table(idx)